# Methods and Results 

In [2]:
# Required packages and versions
# install.packages("janitor")
# install.packages("cowplot")

In [3]:
library(tidyverse)
library(janitor)
library(repr)
library(tidymodels)
library(cowplot)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


ERROR: Error in library(janitor): there is no package called ‘janitor’


## Preliminary Exploratory Data Analysis

#### 1. load the dataset 
Source: https://www.kaggle.com/datasets/risakashiwabara/jpmean-and-standard-deviation-of-height-and-weight/

To answer this question we will analyze a data set of japanese boys' heights and weights. This data, `man.csv` was downloaded from [Kaggle](https://www.kaggle.com/datasets/risakashiwabara/jpmean-and-standard-deviation-of-height-and-weight/)(same as source). Each row in the data set represents an observation (i.e. a japanese boy). 
The columns in the data set `man.csv` represents: 
- `year` - The age of the individual (yrs)
- `category` - The region the individual is from
- `height_average` - The average height of the boys with the same age in this region (cm)
- `height_standard deviation` - The standard deviation of the height of the boys with the same age in this region (cm)
- `body weight _average` - The average weight of the boys with the same age in this region (kg)
- `body weight _standard deviation` - The standard deviation of the weight of the boys with the same age in this region (kg)

In [5]:
man_data <- read_csv("https://raw.githubusercontent.com/Ekenny02/dsci-100-project/main/man.csv")

Rows: 624 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): category
dbl (5): year, height_average, height_standard deviation, body weight _avera...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


### Table 1:  *man_data - untidy*
Here we take a look at the original data set `man_data`, because there are too many observation, we use head and tail to get a sens of what the data set looks like

In [9]:
head(man_data)
tail(man_data)

year,category,height_average,height_standard deviation,body weight _average,body weight _standard deviation
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
5,national,111.0,4.87,19.3,2.79
5,Hokkaido,111.3,4.81,19.3,2.83
5,Aomori,111.8,4.87,19.9,2.78
5,Iwate,111.0,5.08,19.6,3.02
5,Miyagi,111.3,4.90,19.7,3.04
5,Akita,112.3,5.08,19.9,3.16


year,category,height_average,height_standard deviation,body weight _average,body weight _standard deviation
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
17,Nagasaki,170.7,5.51,63.6,10.78
17,Kumamoto,170.6,5.89,62.5,9.30
17,Oita,170.1,5.91,63.3,10.63
17,Miyazaki,170.1,6.13,63.5,11.23
17,Kagoshima,170.4,5.60,62.2,9.88
17,Okinawa,169.5,6.08,62.8,10.65


From the original data set table above, we noticed a few **problems**, which we will address in the data cleaning process: 
1. We notice that some of the column names contain spaces between words. We will clean the column names using `clean_names` to make them consist of only lowercase letters and underscores.
2. In the `category` column, representing the region of each observation, we have identified a category/region labeled "national." This category is the average of the values for all regions in the `category` column.